In [2]:
#import xlrd
import json
import cv2
import subprocess
import os
from collections import Counter
from sklearn.cluster import KMeans
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np

#customize data reading here
pth = "UC San Diego/Sherry Jiang - Instagram Study Subject Data/"
pth2 = "UC\ San\ Diego/Sherry\ Jiang\ -\ Instagram\ Study\ Subject\ Data/"
# bash command: ls -d */ | cut -f1 -d'/' > directories.txt
#sheet = xlrd.open_workbook(pth + 'quatrics_data.xlsx').sheet_by_name('Social+Media+and+Health_January')
#profile_id = [[sheet.cell_value(r, c) for c in range(sheet.ncols)] for r in range(sheet.nrows)]

with open('directories.txt') as f:
    lines = f.readlines()

checked=[]
"""for i in profile_id:
    if i not in lines:
        checked.append(i)
print(checked)"""
# captions, nltk -> polarity 
# image data -> hue, saturation

'for i in profile_id:\n    if i not in lines:\n        checked.append(i)\nprint(checked)'

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
from pathlib import Path
from skimage import color, io
import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [16]:
def analyze(img):
    clf = KMeans(n_clusters = 5)
    color_labels = clf.fit_predict(img)
    center_colors = clf.cluster_centers_
    counts = Counter(color_labels)
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [rgb_to_hex(ordered_colors[i]) for i in counts.keys()]

    plt.figure(figsize = (12, 8))
    plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

    plt.savefig("results/my_pie.png")
    print("Found the following colors:\n")
    for color in hex_colors:
      print(color)

def rgb_to_hex(rgb_color):
    hex_color = "#"
    for i in rgb_color:
        hex_color += ("{:02x}".format(int(i)))
    return hex_color

def preprocess(raw):
    image = cv2.resize(raw, (900, 600), interpolation = cv2.INTER_AREA)                                          
    image = image.reshape(image.shape[0]*image.shape[1], 3)
    return image

In [8]:
##  get_hsv: converts image from rgb to hsv (Hue, Saturation, V=Brightness)
def get_hsv(path):
	if not os.path.exists(Path(path)):
		return 0
	im = io.imread(path)
	im = color.rgb2hsv(im) # http://scikit-image.org/docs/dev/api/skimage.color.html#rgb2hsv
	hsv = []
	for i in range(len(im.shape)):
		if len(im.shape) == 2: #b/w photo
			imdata = im[:,i]
		elif len(im.shape) == 3: #color photo
			imdata = im[:,:,i]
		avg = np.mean(imdata)
		hsv.append( avg )
	return hsv

In [9]:
saved_data = []
num_img, num_posts = 0, 0
for i in lines:
    # emojis -> convert into more meaningful words?
    # loop through all profiles and get the relevant json file
    #p_img = pth2 + 'instagram_data/'+i[:-1]+"/media/posts/"
    p_img = pth + "instagram_data/" + i[:-1] + "/"
    p_text = pth + "instagram_data/" + i[:-1] + "/content/posts_1.json"
    if not os.path.exists(Path(p_text)):
        continue
    #f = open(p_text)
    with open(p_text, 'r') as f:
        content = f.read()
        print(i)
        data = json.loads(content)
    for d in data:
        if not isinstance(d, dict) or 'title' not in d.keys():
            continue
        title = d['title'].encode('latin1').decode('utf8')
        
        ps = sia.polarity_scores(title)
        hsv = [0, 0, 0]
        count = 0
        num_posts += 1
        for m in d['media']:
            if ".jpg" in m["uri"]:
                thsv = get_hsv(p_img + m["uri"])
                if thsv == 0:
                    continue
                hsv = [a + b for a, b in zip(hsv, thsv)]
                count += 1
                num_img += 1
        if count != 0:
            avg_hsv = [x / count for x in hsv]
        saved_data.append([i[:-1], d["creation_timestamp"], title, ps, avg_hsv[0], avg_hsv[1], avg_hsv[2]])
        # creation timestamp | text | polarity score | profile-id -> csv
        # hsv extraction -> output compare with current
        # analysis writeup -> methods used in analysis/feature extraction(text: captions, vader polarity score, number of files; image: hsv, number of images, normalization/mean)
    """cmd = "cd "+p_img + "; ls -d */ | cut -f1"
    dates = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    dates = dates.communicate()[0].strip().decode("utf-8").split("\n")
    p_img = pth + 'instagram_data/'+i[:-1]+"/media/posts/"
    for d in dates:
        file_list=os.listdir(p_img+d[:-1])
        for f in file_list:
            hsv = get_hsv(p_img+d[:-1]+"/"+f)
            print(hsv)
            #image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #modified_image = preprocess(image)
            #analyze(modified_image)"""

R_06CG42nsiYLEn1D

R_11XhwUxcAlfBiUT

R_11atIFjao9ubP3U

R_122PXgLS1mcAwba

R_12PxrvkWKCEVBdO

R_1DongyY81HpyWOS

R_1F97x2pEYk8nIlB

R_1FEuJ2n3SqIjzUy

R_1FLGOK2iocfIzZr

R_1H5ltFuGloJS09W

R_1JEkCHdDwhAtesN

R_1LTOBtRC711VrG5

R_1LZlS6E9feyUpme

R_1LdMJlMSNSaHH6K

R_1N5j4w4z6s2BgHt

R_1NFF3ZnLvZwp2ho

R_1OqKJK2CnGyIRp0

R_1d4x7aJabHIcA2g

R_1d7aAzpVC1BTLkH

R_1f7SoawyRtU35vl

R_1f9spW1WMgmtN83

R_1gGUnNwuk9tgiVP

R_1hHkK4czArVlu35

R_1l3vcDms1XKPffw

R_1mOQndBuIwssLWI

R_1oGWHpMqRNsTZNA

R_1pAs4lX0OzT6rXg

R_1pJxKgIorhVfahc

R_22sNEfPgcoozdU3

R_24GbodB8yAMJZMA

R_24itiXcwjPQi9lm

R_25WJ0CJN3yXcU13

R_27rO7IKcHCwQWfj

R_2BwzCQDRMnky8VC

R_2DSaUGNwjrSgYMk

R_2EgcjWlJe4ghDGV

R_2Ez0T4XWdDwZLLx

R_2EzodmaYdi4UMnL

R_2SAHw9cyS547Me2

R_2Sf6FBMj8Z9WgPX

R_2SizOhIw0onQoRN

R_2Vf5XTcAavimwe9

R_2aCTijBNUL1lWYP

R_2ahs92zJ1xE5SIm

R_2fk7P6uI9MaXlis

R_2fk85QtiMmN2NEs

R_2fvsKU57wqfuw5f

R_2pYIMQA5TBYwDOn

R_2s5855p14qHOPT7

R_2tn8YdhPqgVtbrj

R_2v7AXjA8kzSg7kq

R_2wmXu3tARhAjLJE

R_2wn9TkHpPW

In [10]:
import csv
fields = ['id', 'timestamp', 'caption', 'polarity score', 'hue', 'saturation', 'brightness']
with open("./instagram_posts_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(saved_data)
print(num_img, num_posts)
# print(captions[3].encode('latin1').decode('utf8'))
# cleaning: english language only; 
# image stats (hsv) -> logistic regression

9712 2380


In [39]:
import pandas as pd
import sklearn

x = pd.read_csv("./instagram_posts_data.csv")
survey = pd.read_csv("./survey_with_scores.csv")
print(x.head())
print(y.head())

                  id   timestamp  \
0  R_06CG42nsiYLEn1D  1637559034   
1  R_06CG42nsiYLEn1D  1637072827   
2  R_06CG42nsiYLEn1D  1636325221   
3  R_06CG42nsiYLEn1D  1635715726   
4  R_06CG42nsiYLEn1D  1635131223   

                                         caption  \
0             big little —> formal —> initiation   
1  let’s just say ed and i were both star struck   
2                     quick recap of life so far   
3    halloweekend & kappa kares in one week 🤌🏼🤌🏼   
4                    thumbs up for homecoming 👍🏼   

                                      polarity score       hue  saturation  \
0  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...  0.367593    0.396549   
1  {'neg': 0.2, 'neu': 0.8, 'pos': 0.0, 'compound...  0.399731    0.333645   
2  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...  0.418580    0.313607   
3  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...  0.290845    0.281715   
4  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...  0.305356    0.354468   

 

In [40]:
y = survey[['ResponseId', 'UCLA_LS_mean','UCLA_lonely']]
color_X = x[['id', 'hue', 'saturation', 'brightness']]
color_X = color_X.groupby(['id'])['hue', 'saturation', 'brightness'].mean().reset_index()
color_X.head()

<ipython-input-40-80ce3dd7a6cd>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  color_X = color_X.groupby(['id'])['hue', 'saturation', 'brightness'].mean().reset_index()


,id,hue,saturation,brightness
0,R_06CG42nsiYLEn1D,0.326910,0.293940,0.611632
1,R_11XhwUxcAlfBiUT,0.357403,0.318999,0.524102
2,R_11atIFjao9ubP3U,0.243093,0.322480,0.562066
3,R_122PXgLS1mcAwba,0.182909,0.340807,0.583211
4,R_12PxrvkWKCEVBdO,0.301270,0.334004,0.555513


In [41]:
m = y.ResponseId.isin(color_X.id)
y = y[m]
y = y.sort_values('ResponseId').reset_index()
color_X = color_X.sort_values('id').reset_index()
print(color_X.head())
print(y.head())
# most recent 3/6/12 months' posts -> number of posts -> number of pictures
# binary/multiclass/linear classification

   index                 id       hue  saturation  brightness
0      0  R_06CG42nsiYLEn1D  0.326910    0.293940    0.611632
1      1  R_11XhwUxcAlfBiUT  0.357403    0.318999    0.524102
2      2  R_11atIFjao9ubP3U  0.243093    0.322480    0.562066
3      3  R_122PXgLS1mcAwba  0.182909    0.340807    0.583211
4      4  R_12PxrvkWKCEVBdO  0.301270    0.334004    0.555513
   index         ResponseId  UCLA_LS_mean  UCLA_lonely
0   1010  R_06CG42nsiYLEn1D          2.80          1.0
1   1106  R_11XhwUxcAlfBiUT           NaN          NaN
2    901  R_11atIFjao9ubP3U          2.90          1.0
3    182  R_122PXgLS1mcAwba          2.45          1.0
4    595  R_12PxrvkWKCEVBdO           NaN          NaN


In [42]:
color_X = color_X.drop(index=[60], axis=0).reset_index()
color_X['UCLA_LS_mean'] = y['UCLA_LS_mean'].to_numpy()
color_X['UCLA_lonely'] = y['UCLA_lonely'].to_numpy()
print(color_X.id, y.ResponseId)

0      R_06CG42nsiYLEn1D
1      R_11XhwUxcAlfBiUT
2      R_11atIFjao9ubP3U
3      R_122PXgLS1mcAwba
4      R_12PxrvkWKCEVBdO
             ...        
122    R_tE8wqRqNVOyVYUp
123    R_uwvcWiHnBeLEyC5
124    R_vNeLL90nmJ0A2FX
125    R_xgC1rqRNuYwNpgR
126    R_z0GhR42lxZnEYxP
Name: id, Length: 127, dtype: object 0      R_06CG42nsiYLEn1D
1      R_11XhwUxcAlfBiUT
2      R_11atIFjao9ubP3U
3      R_122PXgLS1mcAwba
4      R_12PxrvkWKCEVBdO
             ...        
122    R_tE8wqRqNVOyVYUp
123    R_uwvcWiHnBeLEyC5
124    R_vNeLL90nmJ0A2FX
125    R_xgC1rqRNuYwNpgR
126    R_z0GhR42lxZnEYxP
Name: ResponseId, Length: 127, dtype: object


In [51]:
color_X = color_X.dropna()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(color_X[['hue', 'saturation', 'brightness']].values, color_X[['UCLA_lonely']].values, test_size=0.25, random_state=0)

In [52]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(x_train, y_train)
pred = clf.predict(x_test)
score = clf.score(x_test, y_test)
print(score)

0.5217391304347826


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [ ]:
saved_data = []
num_comments, num_posts, num_following, num_follower = [], [], [], []
for i in lines:
    # emojis -> convert into more meaningful words?
    # loop through all profiles and get the relevant json file
    #p_img = pth2 + 'instagram_data/'+i[:-1]+"/media/posts/"
    p_comment = pth + "instagram_data/" + i[:-1] + "/comments/post_comments.json"
    p_profilephoto = pth + "instagram_data/" + i[:-1] + "/content/profile_photos.json"
    p_search = pth + "instagram_data/" + i[:-1] + "/recent_searches/word_or_phrases_searches.json"
    p_likedposts = pth + "instagram_data/" + i[:-1] + "/likes/liked_posts.json"
    p_likedcomments = pth + "instagram_data/" + i[:-1] + "/likes/liked_comments.json"
    p_followers = pth + "instagram_data/" + i[:-1] + "/followers_and_following/followers.json"
    p_following = pth + "instagram_data/" + i[:-1] + "/followers_and_following/following.json"
    p_priv = pth + "instagram_data/" + i[:-1] + "/login_and_account_creation/account_privacy_changes.json"

    if not os.path.exists(Path(p_profilephoto)):
        continue
    
    comments = []
    with open(p_comment, 'r') as f:
        content = f.read()
        print(i)
        data = json.loads(content)
    for d in data:
        if not isinstance(d, dict) or 'title' not in d.keys():
            continue
        title = d['title'].encode('latin1').decode('utf8')
        value = d['string_list_data'][0]['value'].encode('latin1').decode('utf8')
        
        #title = d['value'].encode('latin1').decode('utf8')
        l = [i, title, value, d['string_list_data'][0]['timestamp']]
        comments.append(l)
    
    searches = []
    with open(p_search, 'r') as f:
        content = f.read()
        data = json.loads(content)
    for d in data:
        if not isinstance(d, dict) or 'string_list_data' not in d.keys():
            continue
        l = [i, d['string_list_data'][0]['value'], d['string_list_data'][0]['timestamp']]
        searches.append(l)

    with open(p_likedposts, 'r') as f:
        content = f.read()
        data = json.loads(content)
    num_posts.append(len(data))


    with open(p_likedcomments, 'r') as f:
        content = f.read()
        data = json.loads(content)
    num_comments.append(len(data))

    with open(p_followers, 'r') as f:
        content = f.read()
        data = json.loads(content)
    num_follower.append(len(data))


    with open(p_following, 'r') as f:
        content = f.read()
        data = json.loads(content)
    num_following.append(len(data))

    privacy = []
    with open(p_priv, 'r') as f:
        content = f.read()
        data = json.loads(content)
    for d in data:
        if not isinstance(d, dict) or 'title' not in d.keys():
            continue
        l = [i, d['title'], d['string_list_data'][0]['timestamp']]
        privacy.append(l)

In [3]:
saved_data = []
num_comments, num_posts, num_following, num_follower = [], [], [], []

for i in lines:
    p_comment = pth + "instagram_data/" + i[:-1] + "/comments/post_comments.json"

    if not os.path.exists(Path(p_comment)):
        continue
    
    comments = []
    with open(p_comment) as f:
        print(i)
        data = json.load(f)
    #f = open(p_comment)
    #print(type(f))
    #data = json.load(f)
    
    for d in data:
        print(d)
        if not isinstance(d, dict) or 'title' not in d.keys():
            continue
        #title = d['value'].encode('latin1').decode('utf8')
        print('jdiosjfai')
        l = [i, d['title'], d['string_list_data'][0]['value'], d['string_list_data'][0]['timestamp']]
        comments.append(l)
    
    #f.close()

R_06CG42nsiYLEn1D

comments_media_comments
R_11XhwUxcAlfBiUT

comments_media_comments
R_11atIFjao9ubP3U

comments_media_comments
R_122PXgLS1mcAwba

comments_media_comments
R_12PxrvkWKCEVBdO

comments_media_comments
R_1DongyY81HpyWOS

comments_media_comments
R_1F97x2pEYk8nIlB

comments_media_comments
R_1FEuJ2n3SqIjzUy

comments_media_comments
R_1FLGOK2iocfIzZr

comments_media_comments
R_1H5ltFuGloJS09W

comments_media_comments
R_1JEkCHdDwhAtesN

comments_media_comments
R_1LTOBtRC711VrG5

comments_media_comments
R_1LZlS6E9feyUpme

comments_media_comments
R_1LdMJlMSNSaHH6K

comments_media_comments
R_1N5j4w4z6s2BgHt

comments_media_comments
R_1NFF3ZnLvZwp2ho

comments_media_comments
R_1OqKJK2CnGyIRp0

comments_media_comments
R_1d4x7aJabHIcA2g

comments_media_comments
R_1d7aAzpVC1BTLkH

comments_media_comments
R_1f7SoawyRtU35vl

comments_media_comments
R_1f9spW1WMgmtN83

comments_media_comments
R_1gGUnNwuk9tgiVP

comments_media_comments
R_1hHkK4czArVlu35

comments_media_comments
R_1l3vcDms1

In [43]:
saved_data = []
num_comments, num_posts, num_following, num_follower = [], [], [], []
comments = []
for i in lines:
    p_comment = pth + "instagram_data/" + i[:-1] + "/comments/post_comments.json"

    if not os.path.exists(Path(p_comment)):
        continue
    
    
    with open(p_comment, 'r') as f:
        print(i)
        data = json.loads(f.read())
    data = pd.json_normalize(data, record_path =['comments_media_comments'])

    for ind, d in data.iterrows():
        #if not isinstance(d, dict) or 'title' not in d.keys():
        #    continue

        title = d['title'].encode('latin1').decode('utf8')
        value = d['string_list_data'][0]['value'].encode('latin1').decode('utf8')

        l = [i[:-1], title, value, d['string_list_data'][0]['timestamp']]
        comments.append(l)

R_06CG42nsiYLEn1D

R_11XhwUxcAlfBiUT

R_11atIFjao9ubP3U

R_122PXgLS1mcAwba

R_12PxrvkWKCEVBdO

R_1DongyY81HpyWOS

R_1F97x2pEYk8nIlB

R_1FEuJ2n3SqIjzUy

R_1FLGOK2iocfIzZr

R_1H5ltFuGloJS09W

R_1JEkCHdDwhAtesN

R_1LTOBtRC711VrG5

R_1LZlS6E9feyUpme

R_1LdMJlMSNSaHH6K

R_1N5j4w4z6s2BgHt

R_1NFF3ZnLvZwp2ho

R_1OqKJK2CnGyIRp0

R_1d4x7aJabHIcA2g

R_1d7aAzpVC1BTLkH

R_1f7SoawyRtU35vl

R_1f9spW1WMgmtN83

R_1gGUnNwuk9tgiVP

R_1hHkK4czArVlu35

R_1l3vcDms1XKPffw

R_1lmhrr6dMfkIHbv

R_1mOQndBuIwssLWI

R_1oGWHpMqRNsTZNA

R_1pAs4lX0OzT6rXg

R_1pJxKgIorhVfahc

R_22sNEfPgcoozdU3

R_24GbodB8yAMJZMA

R_24itiXcwjPQi9lm

R_25WJ0CJN3yXcU13

R_27rO7IKcHCwQWfj

R_29iAGUnqQU7YgxY

R_2BwzCQDRMnky8VC

R_2DSaUGNwjrSgYMk

R_2EgcjWlJe4ghDGV

R_2Ez0T4XWdDwZLLx

R_2EzodmaYdi4UMnL

R_2SAHw9cyS547Me2

R_2Sf6FBMj8Z9WgPX

R_2SizOhIw0onQoRN

R_2Vf5XTcAavimwe9

R_2YgrTBSGBstn0qY

R_2ZWbUgvLIIiKhiR

R_2aCTijBNUL1lWYP

R_2ahs92zJ1xE5SIm

R_2fk7P6uI9MaXlis

R_2fk85QtiMmN2NEs

R_2fvsKU57wqfuw5f

R_2pYIMQA5TBYwDOn

R_2s5855p14q

In [44]:
import csv
fields = ['id', 'commenter_username', 'comment', 'timestamp']
with open("./instagram_comments_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(comments)
print(len(comments))

86514


In [48]:
saved_data = []
num_comments, num_posts, num_following, num_follower = [], [], [], []
searches = []
for i in lines:
    p_search = pth + "instagram_data/" + i[:-1] + "/recent_searches/word_or_phrase_searches.json"

    if not os.path.exists(Path(p_search)):
        continue
    print(i)
    
    with open(p_search, 'r') as f:
        data = json.loads(f.read())
    data = pd.json_normalize(data, record_path =['searches_keyword'])
    
    print(data)
    for ind, d in data.iterrows():
            #if not isinstance(d, dict) or 'string_list_data' not in d.keys():
            #    continue
        l = [i[:-1], d['string_map_data.Search.value'], d['string_map_data.Time.timestamp']]
        searches.append(l)

R_06CG42nsiYLEn1D

R_122PXgLS1mcAwba

R_1DongyY81HpyWOS

R_1FEuJ2n3SqIjzUy

R_1FLGOK2iocfIzZr

R_1H5ltFuGloJS09W

R_1LZlS6E9feyUpme

R_1N5j4w4z6s2BgHt

R_1d7aAzpVC1BTLkH

R_1f7SoawyRtU35vl

R_1f9spW1WMgmtN83

R_1l3vcDms1XKPffw

R_1oGWHpMqRNsTZNA

R_1pAs4lX0OzT6rXg

R_27rO7IKcHCwQWfj

R_2DSaUGNwjrSgYMk

R_2EzodmaYdi4UMnL

R_2ZWbUgvLIIiKhiR

R_2fvsKU57wqfuw5f

Index(['title', 'string_map_data.æç´¢.href', 'string_map_data.æç´¢.value',
       'string_map_data.æç´¢.timestamp', 'string_map_data.æ¶é´.href',
       'string_map_data.æ¶é´.value', 'string_map_data.æ¶é´.timestamp'],
      dtype='object')
R_2wmXu3tARhAjLJE

R_2xRHgdcgFQz7nFR

R_2zTtZT55KxofdRj

R_32612mE920eA2vT

R_3FWIivKIAxqdY2T

R_3GrMwBz61QxQpSj

R_3JIu4RwDySJQouH

R_3KCHkTJbCLXTzvW

R_3V3gpfHyIqkqFvr

R_3dYA1gfjxafeYoA

R_3gUzOJngxnhLbZ3

R_3ki9nBuQj05tvE4

R_3njB9g0I994WWpk

R_3oNtP3jpQObgVBe

R_3rYS4psVLDrZM6B

R_3sDe8rMEjFDIPAZ

R_5sBALrskhF1TH5n

R_6h7WKwjqMINxeE1

R_9u999IZqmyfJ1aV

R_ABEKq5NwTz5oxB7

R_AujtVX

In [49]:
fields = ['id', 'search_value', 'timestamp']
with open("./instagram_searches_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(searches)
print(len(searches))

189


In [72]:
saved_data = []
for i in lines:

    p_likedposts = pth + "instagram_data/" + i[:-1] + "/likes/liked_posts.json"
    p_likedcomments = pth + "instagram_data/" + i[:-1] + "/likes/liked_comments.json"
    p_followers = pth + "instagram_data/" + i[:-1] + "/followers_and_following/followers.json"
    p_following = pth + "instagram_data/" + i[:-1] + "/followers_and_following/following.json"
    p_priv = pth + "instagram_data/" + i[:-1] + "/login_and_account_creation/account_privacy_changes.json"
    p_acc = pth + "instagram_data/" + i[:-1] + "/account_information/personal_information.json"

    if not os.path.exists(Path(p_acc)):
        continue
    print(i)
    temp = [i[:-1]]
    if not os.path.exists(Path(p_likedposts)):
        temp.append(0)
    else:
        with open(p_likedposts, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['likes_media_likes'])
        temp.append(data.shape[0])

    if not os.path.exists(Path(p_likedcomments)):
        temp.append(0)
    else:
        with open(p_likedcomments, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['likes_comment_likes'])
        temp.append(data.shape[0])

    if not os.path.exists(Path(p_followers)):
        temp.append(0)
    else:
        with open(p_followers, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['relationships_followers'])
        temp.append(data.shape[0])

    if not os.path.exists(Path(p_following)):
        temp.append(0)
    else:
        with open(p_following, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['relationships_following'])
        temp.append(data.shape[0])

    with open(p_acc, 'r') as f:
        data = json.loads(f.read())
    data = pd.json_normalize(data, record_path =['profile_user'])
    temp.append(data.iloc[0][-1])

    temp.append([])
    temp.append([])
    if not os.path.exists(Path(p_priv)):
        continue

    with open(p_priv, 'r') as f:
        data = json.loads(f.read())
    data = pd.json_normalize(data, record_path =['account_history_account_privacy_history'])
    data = data.set_axis(['title', 'href', 'value', 'timestamp'], axis=1, inplace=False)
    df = data.groupby('title')['timestamp'].apply(list).reset_index(name='new')
    temp2 = df['new'].tolist()
    
    if "Switched to Public" not in pd.unique(data['title']) and "Switched to Private" in pd.unique(data['title']):
        temp[6] = temp2[0]
    elif "Switched to Private" in pd.unique(data['title']):
        temp[7] = temp2[1]
        if "Switched to Public" in pd.unique(data['title']):
            temp[6] = temp2[0]
    saved_data.append(temp)


R_06CG42nsiYLEn1D

R_11XhwUxcAlfBiUT

R_11atIFjao9ubP3U

R_122PXgLS1mcAwba

R_12PxrvkWKCEVBdO

R_1DongyY81HpyWOS

R_1F97x2pEYk8nIlB

R_1FEuJ2n3SqIjzUy

R_1FLGOK2iocfIzZr

R_1H5ltFuGloJS09W

R_1JEkCHdDwhAtesN

R_1LTOBtRC711VrG5

R_1LZlS6E9feyUpme

R_1LdMJlMSNSaHH6K

R_1N5j4w4z6s2BgHt

R_1NFF3ZnLvZwp2ho

R_1OqKJK2CnGyIRp0

R_1d4x7aJabHIcA2g

R_1d7aAzpVC1BTLkH

R_1f7SoawyRtU35vl

R_1f9spW1WMgmtN83

R_1gGUnNwuk9tgiVP

R_1hHkK4czArVlu35

R_1l3vcDms1XKPffw

R_1lmhrr6dMfkIHbv

R_1mOQndBuIwssLWI

R_1oGWHpMqRNsTZNA

R_1pAs4lX0OzT6rXg

R_1pJxKgIorhVfahc

R_22sNEfPgcoozdU3

R_24GbodB8yAMJZMA

R_24itiXcwjPQi9lm

R_25WJ0CJN3yXcU13

R_27rO7IKcHCwQWfj

R_29iAGUnqQU7YgxY

R_2BwzCQDRMnky8VC

R_2DSaUGNwjrSgYMk

R_2EgcjWlJe4ghDGV

R_2Ez0T4XWdDwZLLx

R_2EzodmaYdi4UMnL

R_2SAHw9cyS547Me2

R_2Sf6FBMj8Z9WgPX

R_2SizOhIw0onQoRN

R_2Vf5XTcAavimwe9

R_2YgrTBSGBstn0qY

R_2ZWbUgvLIIiKhiR

R_2aCTijBNUL1lWYP

R_2ahs92zJ1xE5SIm

R_2fk7P6uI9MaXlis

R_2fk85QtiMmN2NEs

R_2fvsKU57wqfuw5f

R_2pYIMQA5TBYwDOn

R_2s5855p14q

In [74]:
fields = ['id', 'n_liked_post', 'n_liked_comment', 'n_follower', 'n_following', 'account_privacy', 'switch_to_public', 'switch_to_private']
with open("./instagram_account_variable_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(saved_data)
print(len(saved_data))

116


In [43]:
saved_data = []
for i in lines:

    p_likedposts = pth + "instagram_data/" + i[:-1] + "/likes/liked_posts.json"
    p_likedcomments = pth + "instagram_data/" + i[:-1] + "/likes/liked_comments.json"
    p_comments = pth + "instagram_data/" + i[:-1] + "/comments/post_comments.json"
    p_posts = pth + "instagram_data/" + i[:-1] + "/content/posts_1.json"

    print(i)
    if not os.path.exists(Path(p_posts)):
        continue
    temp = [i[:-1]]
    if not os.path.exists(Path(p_likedposts)):
        temp.append(0)
    else:
        with open(p_likedposts, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['likes_media_likes'])
        timelist = []
        for ind, r in data.iterrows():
            timelist.append(int(r[2][0]['timestamp']))
        freq = 0
        for ind in range(len(timelist)-1):
            freq += timelist[ind] - timelist[ind+1]
        freq = freq / len(timelist)-1
        temp.append(freq)

    if not os.path.exists(Path(p_likedcomments)):
        temp.append(0)
    else:
        with open(p_likedcomments, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['likes_comment_likes'])
        timelist = []
        for ind, r in data.iterrows():
            timelist.append(int(r[2][0]['timestamp']))
        freq = 0
        for ind in range(len(timelist)-1):
            freq += timelist[ind] - timelist[ind+1]
        freq = freq / len(timelist)-1
        temp.append(freq)

    if not os.path.exists(Path(p_comments)):
        temp.append(0)
    else:
        with open(p_comments, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['comments_media_comments'])
        timelist = []
        for ind, r in data.iterrows():
            timelist.append(int(r[2][0]['timestamp']))
        freq = 0
        for ind in range(len(timelist)-1):
            freq += timelist[ind] - timelist[ind+1]
        freq = freq / len(timelist)-1
        temp.append(freq)

    if not os.path.exists(Path(p_posts)):
        temp.append(0)
    else:
        with open(p_posts, 'r') as f:
            data = json.loads(f.read())
        timelist = []
        if isinstance(data, list):
            for r in data:
                timelist.append(int(r['media'][0]['creation_timestamp']))
            freq = 0
            for ind in range(len(timelist)-1):
                freq += timelist[ind] - timelist[ind+1]
            freq = freq / len(timelist)-1
        else:
            freq = data['creation_timestamp']
        temp.append(freq)

    saved_data.append(temp)

R_06CG42nsiYLEn1D

R_11XhwUxcAlfBiUT

R_11atIFjao9ubP3U

R_122PXgLS1mcAwba

R_12JFdtN3OaRkvpk

R_12PxrvkWKCEVBdO

R_1CDecDT9HjrBmM9

R_1Cs8fpt36jOgwK9

R_1DHFecnnl6MqWda

R_1DongyY81HpyWOS

R_1F97x2pEYk8nIlB

R_1FEuJ2n3SqIjzUy

R_1FLGOK2iocfIzZr

R_1GZDjGiOxhVwkXH

R_1H5ltFuGloJS09W

R_1Hi89dFmibifFtw

R_1IouPulQsh4BiXx

R_1JEkCHdDwhAtesN

R_1LTOBtRC711VrG5

R_1LZlS6E9feyUpme

R_1LdMJlMSNSaHH6K

R_1Le2Ey4JZeVONxH

R_1M6P08UQroVTM5W

R_1MX9sR3mRsbYnwA

R_1N5j4w4z6s2BgHt

R_1NFF3ZnLvZwp2ho

R_1OqKJK2CnGyIRp0

R_1OwdidASnsjydWn

R_1d4x7aJabHIcA2g

R_1d7aAzpVC1BTLkH

R_1dEM4nI0XcTXoNJ

R_1f7SoawyRtU35vl

R_1f9spW1WMgmtN83

R_1fa1KJQkA9BgwGp

R_1feAxJ5R42SCrio

R_1gGUnNwuk9tgiVP

R_1gTQMrluAAkB7QN

R_1hHkK4czArVlu35

R_1l3vcDms1XKPffw

R_1lmhrr6dMfkIHbv

R_1mOQndBuIwssLWI

R_1me13VutalCa6NE

R_1oGWHpMqRNsTZNA

R_1pAs4lX0OzT6rXg

R_1pJxKgIorhVfahc

R_1pnjBs561ldV7Bq

R_1polnv3e9ZHFlpU

R_1qaGWn05L5YAZqP

R_1rxDiIp4TFWioil

R_22sNEfPgcoozdU3

R_232kfptYUmUBh10

R_24GbodB8yAMJZMA

R_24itiXcwjP

In [44]:
import csv
fields = ['id', 'f_liked_post', 'f_liked_comment', 'f_comments', 'f_posts']
with open("./instagram_account_activity_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(saved_data)
print(len(saved_data))

134
